In [1]:
from pyscf import gto, scf

'''
Use gto.basis.parse_ecp and gto.basis.load_ecp functions to input
user-specified ecp functions.
'''


'\nUse gto.basis.parse_ecp and gto.basis.load_ecp functions to input\nuser-specified ecp functions.\n'

In [2]:
mol = gto.M(atom='''
 Na 0. 0. 0.
 H  0.  0.  1.''',
            basis={'Na':'lanl2dz', 'H':'sto3g'},
            ecp = {'Na':'lanl2dz'})

mol = gto.M(atom='''
 Na 0. 0. 0.
 H  0.  0.  1.''',
            basis={'Na':'lanl2dz', 'H':'sto3g'},
            ecp = {'Na': gto.basis.parse_ecp('''
Na nelec 10
Na ul
0      2.0000000              6.0000000        
1    175.5502590            -10.0000000        
2      2.3365719             -6.0637782        
2      0.7799867             -0.7299393        
Na S
0    243.3605846              3.0000000        
1     41.5764759             36.2847626        
2     13.2649167             72.9304880        
2      0.9764209              6.0123861        
Na P
0   1257.2650682              5.0000000        
1    189.6248810            117.4495683        
2     54.5247759            423.3986704        
2      0.9461106              7.1241813        
''')})


In [3]:
#
# Simple arithmetic expressions can be specified in ecp input
#
mol = gto.M(atom='''
 Na 0. 0. 0.
 H  0.  0.  1.''',
            basis={'Na':'lanl2dz', 'H':'sto3g'},
            ecp = {'Na': gto.basis.parse_ecp('''
Na nelec 10
Na ul
0      2.0000000              6.0000000*np.exp(.5)
1    175.5502590            -10.0000000*np.exp(.5)
2      2.3365719             -6.0637782*np.exp(.5)
2      0.7799867             -0.7299393*np.exp(.5)
Na S
0    243.3605846              3.0000000
1     41.5764759             36.2847626
2     13.2649167             72.9304880
2      0.9764209              6.0123861
Na P
0   1257.2650682              5.0000000
1    189.6248810            117.4495683
2     54.5247759            423.3986704
2      0.9461106              7.1241813
''')})


In [4]:
#
# Burkatzki-Filippi-Dolg pseudo potential and basis are prefixed with "bfd"
#
mol = gto.M(atom='Na 0. 0. 0.; H 0 0 2.',
            basis={'Na':'bfd-vtz', 'H':'ccpvdz'},
            ecp = {'Na':'bfd-pp'})



In [5]:
#
# Input SOC-ECP parameters
# See also relevant introductions in
#  https://people.clarkson.edu/~pchristi/reps.html 
#  http://www.nwchem-sw.org/index.php/ECP
#
# Note the SOC factor 2/(2l+1) has been multiplied in the SO coefficients
#

In [7]:
mol = gto.M(atom='Cu 0. 0. 0.; H 0 0 2.',
            basis={'Cu':'crenbl', 'H':'ccpvdz'},
            ecp = {'Cu': '''
#    exp                     AREP coef              SO coef
Cu nelec 10
Cu ul
2     16.30159950            -4.73227501             .063509
2     49.98759842           -34.06355667             .272004
2    173.02969360           -90.69224548            1.908336
1    651.10559082           -10.26460838            1.591675
Cu S
2      3.70869994           -87.13957977
2      4.51280022           209.05120850
2      5.53380013          -202.30523682
2     10.20059967           154.84190369
1      2.66059995             9.21743488
0     32.17929840             3.18838096
Cu P
2      3.69499993           -19.07518959            -.604602
2      4.45380020            63.05695343            2.245388
2      6.17630005          -127.18070221           -5.454358
2      8.83930016           158.41213989            8.174906
1     14.67029953            -5.66128206           -1.483716
0     30.43350029             5.39882612             .073914
'''})


In [8]:

'''
Save and load Mole object from/to chkfile
'''

from pyscf import gto
from pyscf import lib

mol = gto.M(atom = '''O 0 0 0; H 0 1 0; H 0 0 1''',
            basis = 'ccpvdz')
print('cc-pvdz basis, nbas = %d' % mol.nbas)

#
# h2o.chk is in HDF5 format.  The mol object is stored in string format under
# key '/mol' 
#

cc-pvdz basis, nbas = 11


In [9]:
lib.chkfile.save_mol(mol, 'h2o.chk')
mol = lib.chkfile.load_mol('h2o.chk')


#
# mol.update_from_chk read the mol from chkfile and overwrite the mol
# attributes
#
mol = gto.M(atom = '''O 0 0 0; H 0 1 0; H 0 0 1''',
            basis = 'sto3g')
print('mol.basis s STO-3G. nbas = %d' % mol.nbas)

mol.update_from_chk('h2o.chk')
print('After calling update_from_chk, mol.basis becomes cc-pvdz. nbas = %d' % mol.nbas)



mol.basis s STO-3G. nbas = 5
After calling update_from_chk, mol.basis becomes cc-pvdz. nbas = 11


In [1]:

'''
Input nuclear model
'''

import numpy
from pyscf import gto

mol = gto.Mole()
mol.atom = [[8,(0, 0, 0)], ['h',(0, 1, 0)], ['H',(0, 0, 1)]]

# 0 means point nuclear model (default), 1 or 'G' means Gaussian nuclear model.
# nuclear model can be set globally
mol.nucmod = 'G'
# or specified in a dictionary, like mol.basis
mol.nucmod = {'O': 'G'}  # Use Gaussian nuclear model for oxygen

mol.build()

In [2]:
#
# Attribute nucmod can be initialized as a function. The function should
# take the nuc_charge as input argument and return the charge distribution
# value "zeta".
#
mol.nucmod = {'O': gto.filatov_nuc_mod, 'H': 0}
mol.build()

#
# The default Gaussian nuclear model is Dyall's nuclear model
# See L. Visscher and K. Dyall, At. Data Nucl. Data Tables, 67, 207 (1997)
# If other gaussian nuclear charge distribution are required,
#    rho(r) = nuc_charge * Norm * exp(-zeta * r^2).
# You can update the gaussian exponent zeta of particular atoms with
# mol.set_nuc_mod function
#
mol.set_nuc_mod(0, 2.)


In [1]:
     #
# Author: Qiming Sun <osirpt.sun@gmail.com>
#

'''
Apply SCF and post-SCF methods
'''

from pyscf import gto

mol = gto.M(atom='H 0 0 0; F 0 0 1.1', basis='ccpvdz')

#
# After initializing the molecular object, call .apply method to run SCF or
# post-SCF calculations
#

In [2]:
mf = mol.apply('RHF').run()
ccobj = mol.apply('CCSD').run()
tdobj = mol.apply('TDHF').run(nstates=5)


# Or fist load all pyscf modules (see more examples 1-advanced/002-input_script.py)

from pyscf import __all__

# then apply post-SCF methods

mf = mol.RHF().run()
ccobj = mol.CCSD(frozen=2).run()
tdobj = mol.TDHF().run(nstates=5)



converged SCF energy = -99.9873974403488
converged SCF energy = -99.9873974403488


<class 'pyscf.cc.ccsd.CCSD'> does not have attributes  converged


E(CCSD) = -100.203735067179  E_corr = -0.216337626830176
converged SCF energy = -99.9873974403488
TD-SCF states [0, 3] not converged.
Excited State energies (eV)
[ 9.20986506  9.20986585 13.62834804 24.09608858 24.0960907 ]
converged SCF energy = -99.9873974403489
converged SCF energy = -99.987397440349
E(CCSD) = -100.1458584691831  E_corr = -0.1584610288341313
converged SCF energy = -99.9873974403489
TD-SCF states [0, 3] not converged.
Excited State energies (eV)
[ 9.20986506  9.20986585 13.62834804 24.09608858 24.0960907 ]
